<a href="https://colab.research.google.com/github/Nevary-Codes/Groq-Assignment/blob/main/Aryan_Manchanda_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import json
import requests
from datetime import datetime
from textwrap import shorten

In [ ]:
class ConversationManager:
    def __init__(self, periodic_k=3):
        self.history = []
        self.periodic_k = periodic_k
        self.run_count = 0
        self.summaries = []

    def add_message(self, role, text):
        self.run_count += 1
        self.history.append({
            "role": role,
            "text": text,
            "ts": datetime.utcnow().isoformat()+"Z"
        })

        # Periodic summarization
        if self.periodic_k > 0 and (self.run_count % self.periodic_k) == 0:
            summary = self.summarize_history()
            self.summaries.append({"ts": datetime.utcnow().isoformat()+"Z", "summary": summary})
            self.history = [{"role": "system", "text": f"SUMMARY: {summary}", "ts": datetime.utcnow().isoformat()+"Z"}]
        return True

    def get_history(self):
        return self.history

    def truncate_by_turns(self, n_turns):
        return self.history[-n_turns:] if n_turns > 0 else []

    def truncate_by_chars(self, max_chars):
        if max_chars <= 0:
            return []
        out = []
        total = 0
        for item in reversed(self.history):
            txt = item["text"]
            if total + len(txt) <= max_chars:
                out.append(item)
                total += len(txt)
            else:
                remaining = max_chars - total
                if remaining > 0:
                    out.append({**item, "text": txt[-remaining:]})
                break
        return list(reversed(out))

    def summarize_history(self, num_sentences=3):
        all_text = " ".join([m["text"] for m in self.history])
        sentences = re.split(r'(?<=[.!?])\s+', all_text.strip())
        if not sentences:
            return "(no content)"
        words = re.findall(r'\w+', all_text.lower())
        freq = {}
        for w in words:
            if len(w) > 2:
                freq[w] = freq.get(w, 0) + 1
        scored = []
        for s in sentences:
            s_words = re.findall(r'\w+', s.lower())
            if not s_words:
                continue
            score = sum(freq.get(w, 0) for w in s_words) / len(s_words)
            scored.append((score, s))
        scored.sort(reverse=True, key=lambda x: x[0])
        top_sentences = [s for _, s in scored[:num_sentences]]
        summary = " ".join([s for s in sentences if s in top_sentences])
        return summary or sentences[0]

In [ ]:
from google.colab import userdata
API_KEY = userdata.get('GROQ_API_KEY')
API_URL = "https://api.groq.com/openai/v1/chat/completions"  # OpenAI-compatible endpoint

FUNCTIONS = [
    {
        "name": "extract_user_info",
        "description": "Extracts structured user information from chat text.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "email": {"type": "string"},
                "phone": {"type": "string"},
                "location": {"type": "string"},
                "age": {"type": "integer"}
            },
            "required": []
        }
    }
]

def extract_with_groq(chat_text):
    if API_KEY is None:
        pass
    payload = {
        "model": "llama-3.1-8b-instant",
        "messages": [{"role": "user", "content": chat_text}],
        "functions": FUNCTIONS,
        "function_call": {"name": "extract_user_info"}
    }
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    resp = requests.post(API_URL, headers=headers, json=payload)
    if resp.status_code == 200:
        result = resp.json()
        try:
            func_args = result['choices'][0]['message']['function_call']['arguments']
            return json.loads(func_args)
        except Exception as e:
            print("Error parsing function_call output:", e)
            return {}
    else:
        print("Request failed:", resp.status_code, resp.text)
        return {}


In [ ]:
mgr = ConversationManager(periodic_k=3)

# Example user messages
user_messages = [
    "Hi, I'm Arjun. My email is arjun.k@example.com and I'm 21 years old.",
    "I live in Chennai and my phone is +91 98765 43210.",
    "Hello, my name is Priya. My email is priya.dev@mail.co and I am 25."
]

print("=== Conversation + Extraction Demo ===")
for msg in user_messages:
    # Add user message
    mgr.add_message("user", msg)

    # Extract structured info from message
    data = extract_with_groq(msg)
    # errors = validate_extracted(data)

    # Add assistant confirmation message
    mgr.add_message("assistant", f"Data extracted: {data}")

    print("\nUser message:", msg)
    print("Extracted:", data)
    # print("Validation:", "OK" if not errors else errors)
    print("Current conversation history length:", len(mgr.get_history()))
    print("Last message in history:", mgr.get_history()[-1]["text"])

=== Conversation + Extraction Demo ===


/tmp/ipython-input-525729238.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ts": datetime.utcnow().isoformat()+"Z"



User message: Hi, I'm Arjun. My email is arjun.k@example.com and I'm 21 years old.
Extracted: {'age': 21, 'email': 'arjun.k@example.com', 'location': '', 'name': 'Arjun', 'phone': ''}
Current conversation history length: 2
Last message in history: Data extracted: {'age': 21, 'email': 'arjun.k@example.com', 'location': '', 'name': 'Arjun', 'phone': ''}


/tmp/ipython-input-525729238.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.summaries.append({"ts": datetime.utcnow().isoformat()+"Z", "summary": summary})
/tmp/ipython-input-525729238.py:20: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.history = [{"role": "system", "text": f"SUMMARY: {summary}", "ts": datetime.utcnow().isoformat()+"Z"}]


Request failed: 400 {"error":{"message":"tool call validation failed: parameters for tool extract_user_info did not match schema: errors: [`/age`: expected integer, but got null]","type":"invalid_request_error","code":"tool_use_failed","failed_generation":"\u003cfunction=extract_user_info\u003e {\"name\": \"\", \"phone\": \"+91 98765 43210\", \"age\": null, \"email\": \"\", \"location\": \"Chennai\", \"name\": \"\"}\u003c/function\u003e"}}


User message: I live in Chennai and my phone is +91 98765 43210.
Extracted: {}
Current conversation history length: 2
Last message in history: Data extracted: {}

User message: Hello, my name is Priya. My email is priya.dev@mail.co and I am 25.
Extracted: {'age': 25, 'email': 'priya.dev@mail.co', 'location': '', 'name': 'Priya', 'phone': ''}
Current conversation history length: 1
Last message in history: SUMMARY: Data extracted: {'age': 21, 'email': 'arjun.k@example.com', 'location': '', 'name': 'Arjun', 'phone': ''} I live in Chennai and my phone 